In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 144676 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.22-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.22-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.22-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [2]:
!mkdir -p drive
!google-drive-ocamlfuse drive


In [6]:
import sys
sys.path.insert(0,'drive/word')

In [ ]:
!pip install keras

In [8]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob


**Drive dan dosya alma**

In [9]:
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
 
from pydrive.drive import GoogleDrive
 
from google.colab import auth
 
from oauth2client.client import GoogleCredentials
 
auth.authenticate_user()
 
gauth = GoogleAuth()
 
gauth.credentials = GoogleCredentials.get_application_default()
 
drive = GoogleDrive(gauth)

**xml dosyasını alma**

In [10]:

from google.colab import files
 
file_id = '1OsMnlWkr3tbKdi6XPwRms1b0bdwprc3_'
 
downloaded = drive.CreateFile({'id': file_id})
 
downloaded.GetContentFile('word.xml')
 
import pandas as pd
import xml.etree.ElementTree as ET 

xtree = ET.parse('word.xml')

xroot = xtree.getroot() 

df_cols = ["filename", "category"]

out_df = pd.DataFrame(columns = df_cols)

for node in xroot: 

    s_path = node.attrib.get("file")

    s_tag = node.attrib.get("tag")

    out_df = out_df.append(pd.Series([s_path, s_tag],

                                     index = df_cols), 

                           ignore_index = True)
print(out_df)

              filename  category
0         word/1/1.jpg      self
1         word/1/2.jpg  adhesive
2         word/1/3.jpg   address
3         word/1/4.jpg    labels
4         word/1/5.jpg        36
...                ...       ...
1151  word/12/1152.jpg    TOILET
1152  word/12/1153.jpg        OF
1153  word/12/1154.jpg     ORDER
1154  word/12/1155.jpg       OUT
1155  word/12/1156.jpg     SORRY

[1156 rows x 2 columns]


In [11]:
out_df["category"].value_counts()

of          12
the         11
and         10
The          9
a            9
            ..
yds          1
Closed       1
pleaes       1
59           1
EMMERICH     1
Name: category, Length: 878, dtype: int64

In [12]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation
model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu',use_bias=True, input_shape=(128,32, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1024, activation = 'relu',use_bias=True))
model.add(Dense(878, activation = 'sigmoid',use_bias=True))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 30, 64)       1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 15, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 60480)             0         
_________________________________________________________________
dense (Dense)                (None, 1024)              61932544  
_________________________________________________________________
dense_1 (Dense)              (None, 878)               899950    
Total params: 62,834,286
Trainable params: 62,834,286
Non-trainable params: 0
_________________________________________________________________


In [13]:
from keras.preprocessing.image import ImageDataGenerator, load_img
batch_size=100
train_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.0,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)
#Data augmentation
train_generator = train_datagen.flow_from_dataframe(
    out_df, 
    "/content/drive",
    x_col='filename',
    y_col='category',
    target_size=(128,32),
    class_mode='categorical',
    batch_size=batch_size
)


Found 1156 validated image filenames belonging to 878 classes.


**Model Oluşturma**

In [ ]:
total_train=1156

history = model.fit_generator(
   train_generator, 
   epochs =60, 
   steps_per_epoch=total_train//batch_size,
)

Epoch 1/60
11/11 [==============================] - 14s 1s/step - loss: 0.8177 - accuracy: 0.7348
Epoch 2/60
11/11 [==============================] - 14s 1s/step - loss: 0.8030 - accuracy: 0.7472
Epoch 3/60
11/11 [==============================] - 14s 1s/step - loss: 0.7691 - accuracy: 0.7509
Epoch 4/60
11/11 [==============================] - 14s 1s/step - loss: 0.7175 - accuracy: 0.7784
Epoch 5/60
11/11 [==============================] - 14s 1s/step - loss: 0.7548 - accuracy: 0.7462
Epoch 6/60
11/11 [==============================] - 14s 1s/step - loss: 0.7390 - accuracy: 0.7699
Epoch 7/60
11/11 [==============================] - 14s 1s/step - loss: 0.7577 - accuracy: 0.7557
Epoch 8/60
11/11 [==============================] - 14s 1s/step - loss: 0.7382 - accuracy: 0.7614
Epoch 9/60
11/11 [==============================] - 14s 1s/step - loss: 0.6888 - accuracy: 0.7775
Epoch 10/60
11/11 [==============================] - 14s 1s/step - loss: 0.6591 - accuracy: 0.7850
Epoch 11/60
11/11 [